In [ ]:
# ! pip install git+https://github.com/catalyst-team/catalyst@kittylyst scikit-learn>=0.20 optuna --upgrade

In [ ]:
# ! pip install catalyst==21.02rc0 scikit-learn>=0.20 optuna --upgrade

###  Catalyst 21.xx demo

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_blobs
%matplotlib inline

In [ ]:
from typing import *

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset

from catalyst import dl, metrics, utils

In [ ]:
# make up a dataset
def make_dataset(seed=42, n_samples=int(1e3)):
    np.random.seed(seed)
    random.seed(seed)
    X, y = make_moons(n_samples=n_samples, noise=0.1)

    y = y*2 - 1 # make y be -1 or 1
    return X, y

def visualize_dataset(X, y):
    plt.figure(figsize=(5,5))
    plt.scatter(X[:,0], X[:,1], c=y, s=20, cmap='jet')

# let's create train data
X_train, y_train = make_dataset()
visualize_dataset(X_train, y_train)

In [ ]:
# valid data
X_valid, y_valid = make_dataset(seed=137)
visualize_dataset(X_valid, y_valid)

In [ ]:
# and another train one (why not?)
X_train2, y_train2 = make_dataset(seed=1337)
visualize_dataset(X_train2, y_train2)

In [ ]:
# initialize a model 
# 2-layer neural network
model = nn.Sequential(
    nn.Linear(2, 16), nn.ReLU(), 
    nn.Linear(16, 16), nn.ReLU(), 
    nn.Linear(16, 1)
)
print(model)
# print("number of parameters", len(model.parameters()))

In [ ]:
def visualize_decision_boundary(X, y, model):
    h = 0.25
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Xmesh = np.c_[xx.ravel(), yy.ravel()]
    
    inputs = torch.tensor([list(xrow) for xrow in Xmesh]).float()
    scores = model(inputs)
    
    Z = np.array([s.data > 0 for s in scores])
    Z = Z.reshape(xx.shape)

    fig = plt.figure()
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.show()
    return fig

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

In [ ]:
t1 = TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_train > 0).float())
t2 = TensorDataset(torch.tensor(X_train2).float(), torch.tensor(y_train2 > 0).float())
v1 = TensorDataset(torch.tensor(X_valid).float(), torch.tensor(y_valid > 0).float())

loaders = {
    "train_1": DataLoader(t1, batch_size=32, num_workers=1), 
    "train_2": DataLoader(t2, batch_size=32, num_workers=1), 
    "valid": DataLoader(v1, batch_size=32, num_workers=1), 
}

---

### Act 1 - ``CustomRunner – batch handling by you own``

In [ ]:
# 2-layer neural network
model = nn.Sequential(
    nn.Linear(2, 16), nn.ReLU(), 
    nn.Linear(16, 16), nn.ReLU(), 
    nn.Linear(16, 1)
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
experiment = dl.SingleStageExperiment(
    model=model, 
    optimizer=optimizer, 
    loaders=loaders, 
    num_epochs=5
)

class CustomRunner(dl.IStageBasedRunner):
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)

        loss = F.binary_cross_entropy_with_logits(y_hat.view(-1), y)
        self.batch_metrics = {"loss": loss}
        if self.loader_batch_step % 10 == 0:
            print(
                f"{self.loader_key} ({self.loader_batch_step}/{self.loader_batch_len}:" 
                f"loss {loss.item()}"
            )

        if self.is_train_loader:
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

CustomRunner().run(experiment)

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

---

### Act 2 - ``SupervisedRunner – Runner with Callbacks``

In [ ]:
num_epochs=5
model = nn.Sequential(
    nn.Linear(2, 16), nn.ReLU(), 
    nn.Linear(16, 16), nn.ReLU(), 
    nn.Linear(16, 1)
)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4])

experiment = dl.SingleStageExperiment(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler,
    loaders=loaders, 
    num_epochs=num_epochs,
    callbacks={
# Let's use AUC metric as an example – it's loader-based, so we shouldn't compute it on each batch
        "auc": dl.LoaderMetricCallback(
            metric=metrics.AUCMetric(),
            input_key="scores", target_key="targets", 
        ), 
# To wrap the criterion step logic, you could use CriterionCallback:
        "criterion": dl.CriterionCallback(
            metric_key="loss", 
            input_key="logits", 
            target_key="targets"
        ), 
# To wrap the optimizer step logic, you could use OptimizerCallback:
        "optimizer": dl.OptimizerCallback(metric_key="loss"), 
# The same case with the scheduler:
        "scheduler": dl.SchedulerCallback(
            loader_key="valid", metric_key="loss"
        ),
# We could also use lrfinder for lr scheduling:
#         "lr-finder": dl.LRFinder(
#             final_lr=1.0,
#             scale="log",
#             num_steps=None,
#             optimizer_key=None,
#         ),
# You can select any number of metrics to checkpoint on:
        "checkpoint1": dl.CheckpointCallback(
            logdir="./logdir2/auc",
            loader_key="valid", metric_key="auc", 
            minimize=False, save_n_best=3
        ),
        "checkpoint2": dl.CheckpointCallback(
            logdir="./logdir2/loss",
            loader_key="valid", metric_key="loss", 
            minimize=True, save_n_best=1
        ),
# Or turn on/off tqdm verbose during loader run:
        "verbose": dl.VerboseCallback(),
    },
    loggers={
        "console": dl.ConsoleLogger(),
#         "csv": dl.LogdirLogger(logdir="./logdir2"),
        "tensorboard": dl.TensorboardLogger(logdir="./logdir2/tb"),
    }
)

class CustomSupervisedRunner(dl.IStageBasedRunner):
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

CustomSupervisedRunner().run(experiment)

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

---

### Act 3 - ``CustomMetric``

In [ ]:
num_epochs=5
model = nn.Sequential(
    nn.Linear(2, 16), nn.ReLU(), 
    nn.Linear(16, 16), nn.ReLU(), 
    nn.Linear(16, 1)
)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4])


class CustomAccuracyMetric(metrics.ICallbackBatchMetric, metrics.AdditiveValueMetric):
    def update(self, scores: torch.Tensor, targets: torch.Tensor) -> float:
        value = ((scores > 0.5) == targets).float().mean().item()
        value = super().update(value, len(targets))
        return value
    
    def update_key_value(self, scores: torch.Tensor, targets: torch.Tensor) -> Dict[str, float]:
        value = self.update(scores, targets)
        return {"accuracy": value}

    def compute_key_value(self) -> Dict[str, float]:
        mean, std = super().compute()
        return {"accuracy": mean, "accuracy/std": std}

    
experiment = dl.SingleStageExperiment(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler,
    loaders=loaders, 
    num_epochs=num_epochs,
    callbacks={
        "accuracy": dl.BatchMetricCallback(
            metric=CustomAccuracyMetric(), log_on_batch=True,
            input_key="scores", target_key="targets", 
        ),
        "auc": dl.LoaderMetricCallback(
            metric=metrics.AUCMetric(),
            input_key="scores", target_key="targets", 
        ), 
        "criterion": dl.CriterionCallback(
            metric_key="loss", 
            input_key="logits", 
            target_key="targets"
        ), 
        "optimizer": dl.OptimizerCallback(metric_key="loss"), 
        "scheduler": dl.SchedulerCallback(
            loader_key="valid", metric_key="loss"
        ),
        "checkpoint1": dl.CheckpointCallback(
            logdir="./logdir3/accuracy",
            loader_key="valid", metric_key="accuracy", 
            minimize=False, save_n_best=3
        ),
        "checkpoint2": dl.CheckpointCallback(
            logdir="./logdir3/loss",
            loader_key="valid", metric_key="loss", 
            minimize=True, save_n_best=1
        ),
#         "verbose": dl.VerboseCallback(),
    },
    loggers={
        "console": dl.ConsoleLogger(),
        "tensorboard": dl.TensorboardLogger(logdir="./logdir3/tb"),
    }
)

class CustomSupervisedRunner(dl.IStageBasedRunner):
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

CustomSupervisedRunner().run(experiment)

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

---

### Act 4 - ``CustomCallback``

In [ ]:
num_epochs=5
model = nn.Sequential(
    nn.Linear(2, 16), nn.ReLU(), 
    nn.Linear(16, 16), nn.ReLU(), 
    nn.Linear(16, 1)
)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4])


# Let's plot the decision doundary after each epoch:
class VisualizationCallback(dl.Callback):
    def __init__(self):
        super().__init__(order=dl.CallbackOrder.External)

    def on_epoch_end(self, runner):
        img = visualize_decision_boundary(X_valid, y_valid, runner.model)


experiment = dl.SingleStageExperiment(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler,
    loaders=loaders, 
    num_epochs=num_epochs,
    callbacks={
        "criterion": dl.CriterionCallback(
            metric_key="loss", 
            input_key="logits", 
            target_key="targets"
        ), 
        "optimizer": dl.OptimizerCallback(metric_key="loss"), 
        "scheduler": dl.SchedulerCallback(
            loader_key="valid", metric_key="loss"
        ),
        "checkpoint": dl.CheckpointCallback(
            logdir="./logdir4/loss",
            loader_key="valid", metric_key="loss", 
            minimize=True, save_n_best=1
        ),
# And include it into callbacks:        
        "visualization": VisualizationCallback()
    },
    loggers={
        "console": dl.ConsoleLogger(),
        "tensorboard": dl.TensorboardLogger(logdir="./logdir4/tb"),
    }
)

class CustomSupervisedRunner(dl.IStageBasedRunner):
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

CustomSupervisedRunner().run(experiment)

---

### Act 5 - ``CustomLogger``

In [ ]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt

def get_img_from_fig(fig, dpi=180):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
num_epochs=5
model = nn.Sequential(
    nn.Linear(2, 16), nn.ReLU(), 
    nn.Linear(16, 16), nn.ReLU(), 
    nn.Linear(16, 1)
)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4])


# We need to add only a few lines to log the image to all runner's loggers
class VisualizationCallback(dl.Callback):
    def __init__(self):
        super().__init__(order=dl.CallbackOrder.External)

    def on_epoch_end(self, runner):
        image = visualize_decision_boundary(X_valid, y_valid, runner.model)
        image = get_img_from_fig(image)
        # runner will propagate it to all loggers
        runner.log_image(tag="decision_boundary", image=image, scope="epoch")


# Let's also add our own Logger to store image on the disk
class VisualizationLogger(dl.ILogger):
    def __init__(self, logdir: str):
        self.logdir = logdir
        os.makedirs(self.logdir, exist_ok=True)
        
    def log_image(
        self,
        tag: str,
        image: np.ndarray,
        scope: str = None,
        # experiment info
        experiment_key: str = None,
        global_epoch_step: int = 0,
        global_batch_step: int = 0,
        global_sample_step: int = 0,
        # stage info
        stage_key: str = None,
        stage_epoch_len: int = 0,
        stage_epoch_step: int = 0,
        stage_batch_step: int = 0,
        stage_sample_step: int = 0,
        # loader info
        loader_key: str = None,
        loader_batch_len: int = 0,
        loader_sample_len: int = 0,
        loader_batch_step: int = 0,
        loader_sample_step: int = 0,
    ) -> None:
        if scope == "epoch":
            plt.imsave(
                os.path.join(self.logdir, f"{tag}_{stage_key}_{stage_epoch_step}.png"),
                image,
            )


experiment = dl.SingleStageExperiment(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler,
    loaders=loaders, 
    num_epochs=num_epochs,
    callbacks={
        "criterion": dl.CriterionCallback(
            metric_key="loss", 
            input_key="logits", 
            target_key="targets"
        ), 
        "optimizer": dl.OptimizerCallback(metric_key="loss"), 
        "scheduler": dl.SchedulerCallback(
            loader_key="valid", metric_key="loss"
        ),
        "checkpoint": dl.CheckpointCallback(
            logdir="./logdir5/loss",
            loader_key="valid", metric_key="loss", 
            minimize=True, save_n_best=1
        ),
        "visualization": VisualizationCallback()
    },
    loggers={
        "console": dl.ConsoleLogger(),
        "visualization": VisualizationLogger(logdir="./logdir5/visualization"),
        "tensorboard": dl.TensorboardLogger(logdir="./logdir5/tb"),
    }
)

class CustomSupervisedRunner(dl.IStageBasedRunner):
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

CustomSupervisedRunner().run(experiment)

In [ ]:
! ls ./logdir5
! ls ./logdir5/loss
! ls ./logdir5/tb
! ls ./logdir5/visualization

### Act 6 - ``Multistage Experiment``

In [ ]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt

def get_img_from_fig(fig, dpi=180):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
loaders = {
    "stage_1": {
        "train_1": DataLoader(t1, batch_size=32, num_workers=1), 
        "valid": DataLoader(v1, batch_size=32, num_workers=1), 
    },
    "stage_2": {
        "train_2": DataLoader(t2, batch_size=32, num_workers=1), 
        "valid": DataLoader(v1, batch_size=32, num_workers=1), 
    },
}
stages = loaders.keys()
num_epochs=5

model = nn.Sequential(
    nn.Linear(2, 16), nn.ReLU(), 
    nn.Linear(16, 16), nn.ReLU(), 
    nn.Linear(16, 1)
)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4])


# let's define our own Experiment logic
# to unpack key-value loaders into a stages-based loaders:
class CustomExperiment(dl.SingleStageExperiment):
    @property
    def stages(self):
        return self._stage

    def get_loaders(self, stage: str):
        return self._loaders[stage]

    
class VisualizationCallback(dl.Callback):
    def __init__(self):
        super().__init__(order=dl.CallbackOrder.External)

    def on_epoch_end(self, runner):
        image = visualize_decision_boundary(X_valid, y_valid, runner.model)
        image = get_img_from_fig(image)
        # runner will propagate it to all loggers
        runner.log_image(tag="decision_boundary", image=image, scope="epoch")


class VisualizationLogger(dl.ILogger):
    def __init__(self, logdir: str):
        self.logdir = logdir
        os.makedirs(self.logdir, exist_ok=True)
        
    def log_image(
        self,
        tag: str,
        image: np.ndarray,
        scope: str = None,
        # experiment info
        experiment_key: str = None,
        global_epoch_step: int = 0,
        global_batch_step: int = 0,
        global_sample_step: int = 0,
        # stage info
        stage_key: str = None,
        stage_epoch_len: int = 0,
        stage_epoch_step: int = 0,
        stage_batch_step: int = 0,
        stage_sample_step: int = 0,
        # loader info
        loader_key: str = None,
        loader_batch_len: int = 0,
        loader_sample_len: int = 0,
        loader_batch_step: int = 0,
        loader_sample_step: int = 0,
    ) -> None:
        if scope == "epoch":
            plt.imsave(
                os.path.join(self.logdir, f"{tag}_{stage_key}_{stage_epoch_step}.png"),
                image,
            )


experiment = CustomExperiment(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler,
    loaders=loaders, 
    num_epochs=num_epochs,
    callbacks={
        "auc": dl.LoaderMetricCallback(
            metric=metrics.AUCMetric(),
            input_key="scores", target_key="targets", 
        ), 
        "criterion": dl.CriterionCallback(
            metric_key="loss", 
            input_key="logits", 
            target_key="targets"
        ), 
        "optimizer": dl.OptimizerCallback(metric_key="loss"), 
        "scheduler": dl.SchedulerCallback(
            loader_key="valid", metric_key="loss"
        ),
        "checkpoint1": dl.CheckpointCallback(
            logdir="./logdir6/auc",
            loader_key="valid", metric_key="auc", 
            minimize=False, save_n_best=3
        ),
        "checkpoint2": dl.CheckpointCallback(
            logdir="./logdir6/loss",
            loader_key="valid", metric_key="loss", 
            minimize=True, save_n_best=1
        ),
        "visualization": VisualizationCallback(),
#         "verbose": VerboseCallback(),
        
    },
    loggers={
        "console": dl.ConsoleLogger(),
        "visualization": VisualizationLogger(logdir="./logdir6/visualization"),
        "tensorboard": dl.TensorboardLogger(logdir="./logdir6/tb"),
    },
# here is the trick for multi-stage support:
    stage=stages
)

class CustomRunner(dl.IStageBasedRunner):
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

CustomRunner().run(experiment)

In [ ]:
! ls ./logdir6

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

---

### Act 8 - ``CustomExperiment``

In [ ]:
class CustomExperiment(dl.IExperiment):
    @property
    def seed(self) -> int:
        return 73

    @property
    def name(self) -> str:
        return "experiment73"

    @property
    def hparams(self) -> Dict:
        return {}

    @property
    def stages(self) -> List[str]:
        return ["stage_1", "stage_2"]

    def get_stage_params(self, stage: str) -> Dict[str, Any]:
        if stage == "stage_1":
            return {
                "num_epochs": 10,
                "migrate_model_from_previous_stage": False,
                "migrate_callbacks_from_previous_stage": False,
            }
        elif stage == "stage_2":
            return {
                "num_epochs": 6,
                "migrate_model_from_previous_stage": True,
                "migrate_callbacks_from_previous_stage": False,
            }
        else:
            raise NotImplemented()

    def get_loaders(self, stage: str) -> Dict[str, Any]:
        if stage == "stage_1":
            return {
                "train_1": DataLoader(t1, batch_size=32, num_workers=1), 
                "valid": DataLoader(v1, batch_size=32, num_workers=1), 
            }
        elif stage == "stage_2":
            return {
                "train_2": DataLoader(t2, batch_size=32, num_workers=1), 
                "valid": DataLoader(v1, batch_size=32, num_workers=1), 
            }
        else:
            raise NotImplemented()

    def get_model(self, stage: str):
        return nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), 
            nn.Linear(16, 16), nn.ReLU(), 
            nn.Linear(16, 1)
        )

    def get_criterion(self, stage: str):
        return nn.BCEWithLogitsLoss()

    def get_optimizer(self, stage: str, model):
        if stage == "stage_1":
            return torch.optim.Adam(model.parameters(), lr=0.02)
        elif stage == "stage_2":
            return torch.optim.SGD(model.parameters(), lr=0.01)
        else:
            raise NotImplemented()

    def get_scheduler(self, stage: str, optimizer):
        if stage == "stage_1":
            return torch.optim.lr_scheduler.MultiStepLR(optimizer, [3, 8])
        elif stage == "stage_2":
            return torch.optim.lr_scheduler.MultiStepLR(optimizer, [3, 6])
        else:
            raise NotImplemented()
        

    def get_callbacks(self, stage: str) -> Dict[str, dl.Callback]:
        if stage == "stage_1":
            
            return {
                "criterion": dl.CriterionCallback(
                    metric_key="loss", 
                    input_key="logits", 
                    target_key="targets"
                ), 
                "optimizer": dl.OptimizerCallback(metric_key="loss"), 
                "scheduler": dl.SchedulerCallback(
                    loader_key="valid", metric_key="loss"
                ),
                "checkpoint": dl.CheckpointCallback(
                    logdir="./logdir7/loss",
                    loader_key="valid", metric_key="loss", 
                    minimize=True, save_n_best=3
                ),
            }
        elif stage == "stage_2":
            return {
                "auc": dl.LoaderMetricCallback(
                    metric=metrics.AUCMetric(),
                    input_key="scores", target_key="targets", 
                ), 
                "criterion": dl.CriterionCallback(
                    metric_key="loss", 
                    input_key="logits", 
                    target_key="targets"
                ), 
                "optimizer": dl.OptimizerCallback(metric_key="loss"), 
                "scheduler": dl.SchedulerCallback(
                    loader_key="valid", metric_key="loss"
                ),
                "checkpoint_auc": dl.CheckpointCallback(
                    logdir="./logdir7/auc",
                    loader_key="valid", metric_key="auc", 
                    minimize=False, save_n_best=3
                ),
            }
        else:
            raise NotImplemented()
        

    def get_engine(self):
        if torch.cuda.is_available():
            return dl.DeviceEngine("cuda:0")
        else:
            return dl.DeviceEngine("cpu")

    def get_trial(self):
        return None

    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
            "tensorboard": dl.TensorboardLogger(logdir="./logdir7/tb"),
        }

    
class CustomRunner(dl.IStageBasedRunner):
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }
    
experiment = CustomExperiment()
CustomRunner().run(experiment)

---

### Act 8 - integration with hyperparameter search

In [ ]:
from datetime import datetime
import optuna    

class CustomRunner(dl.IStageBasedRunner):
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

def objective(trial):
    num_epochs = 6
    num_hidden1 = int(trial.suggest_loguniform("num_hidden1", 2, 16))
    num_hidden2 = int(trial.suggest_loguniform("num_hidden2", 2, 16))
    
    loaders = {
        "train_1": DataLoader(t1, batch_size=32, num_workers=1), 
        "train_2": DataLoader(t2, batch_size=32, num_workers=1), 
        "valid": DataLoader(v1, batch_size=32, num_workers=1), 
    }

    model = nn.Sequential(
        nn.Linear(2, num_hidden1), nn.ReLU(), 
        nn.Linear(num_hidden1, num_hidden2), nn.ReLU(), 
        nn.Linear(num_hidden2, 1)
    )
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [3, 6])

    logdir = f"./logdir8/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    experiment = dl.SingleStageExperiment(
        model=model, 
        criterion=criterion, 
        optimizer=optimizer, 
        scheduler=scheduler,
        loaders=loaders, 
        num_epochs=num_epochs,
        callbacks={
            "auc": dl.LoaderMetricCallback(
                metric=metrics.AUCMetric(),
                input_key="scores", target_key="targets", 
            ), 
            "criterion": dl.CriterionCallback(
                metric_key="loss", 
                input_key="logits", 
                target_key="targets"
            ), 
            "optimizer": dl.OptimizerCallback(metric_key="loss"), 
            "scheduler": dl.SchedulerCallback(
                loader_key="valid", metric_key="loss"
            ),
            "checkpoint": dl.CheckpointCallback(
                logdir=f"{logdir}/auc",
                loader_key="valid", metric_key="auc", 
                minimize=False, save_n_best=3
            ),
            "optuna": dl.OptunaPruningCallback(loader_key="valid", metric_key="auc")
        },
        loggers={
            "console": dl.ConsoleLogger(),
            "tensorboard": dl.TensorboardLogger(logdir=f"{logdir}/tb"),
        },
        trial=trial,
    )

    runner = CustomRunner()
    runner.run(experiment)
    score = runner.callbacks["checkpoint"].top_best_metrics[0][0]
    
    return score

study = optuna.create_study(
    direction="maximize",
#     direction="minimize",
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=0, n_warmup_steps=0, interval_steps=1
    ),
)
study.optimize(objective, n_trials=5, timeout=300)
print(study.best_value, study.best_params)

---

### Act 9 - Confusion Matrix logging - IMetric+ICallback+ILogger

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from catalyst import dl, metrics, utils

# sample data
num_samples, num_features, num_classes = int(1e4), int(1e1), 6
X = torch.rand(num_samples, num_features)
y = (torch.rand(num_samples, ) * num_classes).to(torch.int64)

# pytorch loaders
dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=32, num_workers=1)
loaders = {"train": loader, "valid": loader}

# model, criterion, optimizer, scheduler
model = torch.nn.Linear(num_features, num_classes)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [2])

num_epochs = 6
experiment = dl.SingleStageExperiment(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler,
    loaders=loaders, 
    num_epochs=num_epochs,
    callbacks={
        "accuracy": dl.BatchMetricCallback(
            metric=metrics.AccuracyMetric(num_classes=num_classes),
            input_key="probs", target_key="targets", 
        ),
        "auc": dl.LoaderMetricCallback(
            metric=metrics.AUCMetric(),
            input_key="scores", target_key="targets", 
        ), 
        "criterion": dl.CriterionCallback(
            metric_key="loss", 
            input_key="logits", 
            target_key="targets",
        ), 
        "optimizer": dl.OptimizerCallback(metric_key="loss"), 
        "scheduler": dl.SchedulerCallback(
            loader_key="valid", metric_key="loss"
        ),
        "checkpoint1": dl.CheckpointCallback(
            logdir="./logdir9/loss",
            loader_key="valid", metric_key="loss", 
            minimize=False, save_n_best=3
        ),
        "checkpoint2": dl.CheckpointCallback(
            logdir="./logdir9/auc",
            loader_key="valid", metric_key="auc", 
            minimize=True, save_n_best=1
        ),
        "checkpoint3": dl.CheckpointCallback(
            logdir="./logdir9/accuracy",
            loader_key="valid", metric_key="accuracy", 
            minimize=True, save_n_best=1
        ),
        "verbose": dl.VerboseCallback(),
        "confusion_matrix": dl.ConfusionMatrixCallback(
            input_key="probs", 
            target_key="targets",
            prefix="confusion_matrix",
            num_classes=num_classes,
        )
    },
    loggers={
        "console": dl.ConsoleLogger(),
        "csv": dl.CSVLogger(logdir="./logdir9"),
        "tensorboard": dl.TensorboardLogger(logdir="./logdir9/tb"),
    }
)

class CustomSupervisedRunner(dl.IStageBasedRunner):
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat,
            "scores": torch.sigmoid(y_hat),
            "probs": torch.softmax(y_hat, dim=1),
        }

CustomSupervisedRunner().run(experiment)

---

In [ ]:
from catalyst import dl, metrics, utils

### Act 10 - Minimal example with SupervisedRunner and linear regression

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from catalyst import dl


# data
num_samples, num_features = int(1e4), int(1e1)
X, y = torch.rand(num_samples, num_features), torch.rand(num_samples)
dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=32, num_workers=1)
loaders = {"train": loader, "valid": loader}

# model, criterion, optimizer, scheduler
model = torch.nn.Linear(num_features, 1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [3, 6])

# model training
runner = dl.SupervisedRunner()
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    logdir="./logdir10",
    num_epochs=8,
    verbose=True,
    # @TODO: automate
    loggers={
        "console": dl.ConsoleLogger(),
        "csv": dl.CSVLogger(logdir="./logdir10"),
        "tensorboard": dl.TensorboardLogger(logdir="./logdir10/tb"),
    }
)

---

### Act 11 - Minimal example with SupervisedRunner and multiclass classification

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from catalyst import dl, metrics, utils

# sample data
num_samples, num_features, num_classes = int(1e4), int(1e1), 4
X = torch.rand(num_samples, num_features)
y = (torch.rand(num_samples, ) * num_classes).to(torch.int64)

# pytorch loaders
dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=32, num_workers=1)
loaders = {"train": loader, "valid": loader}

# model, criterion, optimizer, scheduler
model = torch.nn.Linear(num_features, num_classes)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [2])

# model training
runner = dl.SupervisedRunner()
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    logdir="./logdir11",
    num_epochs=3,
    verbose=True,
#     callbacks=[dl.AccuracyCallback(num_classes=num_classes)]
    callbacks={
        "classification": dl.LoaderMetricCallback(
            metric=metrics.PrecisionRecallF1SupportMetric(num_classes=num_classes, mode="multiclass"),
            input_key="logits", target_key="targets", 
        ),
    },
    # @TODO: automate
    loggers={
        "console": dl.ConsoleLogger(),
        "csv": dl.CSVLogger(logdir="./logdir11"),
        "tensorboard": dl.TensorboardLogger(logdir="./logdir11/tb"),
    }
)

----

### Act 12 - Minimal example with SupervisedRunner and multilabel classification

In [ ]:
# @TODO

---

### Act 13 - Minimal example with SupervisedRunner and image classification/segmentation

In [ ]:
import os
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from catalyst import dl, metrics
from catalyst.contrib.data.cv import ToTensor
from catalyst.contrib.datasets import MNIST

class ClassifyUnet(nn.Module):

    def __init__(self, in_channels, in_hw, out_features):
        super().__init__()
        self.encoder = nn.Sequential(nn.Conv2d(in_channels, in_channels, 3, 1, 1), nn.Tanh())
        self.decoder = nn.Conv2d(in_channels, in_channels, 3, 1, 1)
        self.clf = nn.Linear(in_channels * in_hw * in_hw, out_features)

    def forward(self, x):
        z = self.encoder(x)
        z_ = z.view(z.size(0), -1)
        y_hat = self.clf(z_)
        x_ = self.decoder(z)
        return y_hat, x_

model = ClassifyUnet(1, 28, 10)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)

loaders = {
    "train": DataLoader(MNIST(os.getcwd(), train=True, download=True, transform=ToTensor()), batch_size=32),
    "valid": DataLoader(MNIST(os.getcwd(), train=False, download=True, transform=ToTensor()), batch_size=32),
}

class CustomRunner(dl.Runner):

    def handle_batch(self, batch):
        x, y = batch
        x_noise = (x + torch.rand_like(x)).clamp_(0, 1)
        y_hat, x_ = self.model(x_noise)
        
        self.batch = {
#             "image": x,
            "clf_targets": y,
            "seg_targets": x,
            "clf_logits": y_hat,
            "seg_logits": x_,
        }


runner = CustomRunner()
runner.train(
    loaders=loaders, 
    model=model, 
    criterion=criterion,
    optimizer=optimizer, 
    logdir="./logdir13",
    num_epochs=3,
    verbose=True,
    callbacks={
        "classification": dl.LoaderMetricCallback(
            metric=metrics.PrecisionRecallF1SupportMetric(num_classes=10, mode="multiclass"),
            input_key="clf_logits", target_key="clf_targets", 
        ),
        "segmentation": dl.BatchMetricCallback(
            metric=metrics.IOUMetric(),
            input_key="seg_logits", target_key="seg_targets", 
        ),
        "criterion": dl.CriterionCallback(
            metric_key="loss", 
            input_key="clf_logits", 
            target_key="clf_targets",
        ), 
        "optimizer": dl.OptimizerCallback(metric_key="loss"), 
    },
    # @TODO: automate
    loggers={
        "console": dl.ConsoleLogger(),
        "csv": dl.CSVLogger(logdir="./logdir13"),
        "tensorboard": dl.TensorboardLogger(logdir="./logdir13/tb"),
    }
)

----

🎉 You have passed ``Kittylyst`` tutorial! This is just a minimal educational demo, but I hope you found it interesting for your deep learning research code organisation.

For more advanced and production-ready solution please follow our [Catalyst](https://github.com/catalyst-team/catalyst) repository.

PS. If you are interested in deep learning you could also try out our [dl-course](https://github.com/catalyst-team/dl-course).